In [1]:
from transformers import pipeline

In [2]:
tqa = pipeline(task="table-question-answering", model="google/tapas-large-finetuned-wtq")

In [3]:
import tabula
import PyPDF4
import os

base_path = 'test_tables'

# Specify the path to your PDF file
pdf_path = "/path/to/sample_table.pdf"

def get_pdf_page_count(pdf_path):
    with open(pdf_path, "rb") as f:
        reader = PyPDF4.PdfFileReader(f)
        page_count = reader.numPages
    return page_count

total_pages = get_pdf_page_count(pdf_path)

for page_num in range(1, total_pages+1):
    
    tables = tabula.read_pdf(pdf_path, pages=str(page_num), multiple_tables=True)
    
    page_path = os.path.join(base_path, str(page_num))
    
    if not os.path.exists(page_path):
        os.makedirs(page_path)
    
    for idx, table in enumerate(tables):
        table.to_csv('{}/{}/table_{}.csv'.format(base_path,page_num,str(idx+1)), index = False)

In [5]:
import pandas as pd

directory = '{}/{}'.format(base_path,str(2))

# Initialize an empty dictionary to store dataframes
dfs = []

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file into a dataframe and store it in the dictionary
        dfs.append(pd.read_csv(file_path))

In [6]:
dfs

[   Time (drops of water)  Distance (cm)
 0                      1        10,11,9
 1                      2     29, 31, 30
 2                      3     59, 58, 61
 3                      4   102, 100, 98
 4                      5  122, 125, 127,
     Speed (mph)           Driver                         Car     Engine  \
 0       407.447  Craig Breedlove           Spirit of America     GE J47   
 1       413.199        Tom Green            Wingfoot Express     WE J46   
 2       434.220       Art Arfons               Green Monster     GE J79   
 3       468.719  Craig Breedlove           Spirit of America     GE J79   
 4       526.277  Craig Breedlove           Spirit of America     GE J79   
 5       536.712       Art Arfons               Green Monster     GE J79   
 6       555.127  Craig Breedlove  Spirit of America, Sonic 1     GE J79   
 7       576.553       Art Arfons               Green Monster     GE J79   
 8       600.601  Craig Breedlove  Spirit of America, Sonic 1     GE 

In [8]:
type(dfs[1])

pandas.core.frame.DataFrame

In [11]:
table_answer = tqa(table=dfs[1].astype(str), query="What is the engine of spirit of america car?")['cells'][0]

In [12]:
table_answer

'GE J47'